In [ ]:
from neo4j import GraphDatabase, basic_auth
import pandas as pd


driver = GraphDatabase.driver(
  "bolt://18.212.185.199:7687",
  auth=basic_auth("neo4j", "companies-superintendent-chit"))

1. i nomi delle squadre che hanno partecipano al torneo nel 2015

In [ ]:
query = '''
MATCH (a:Team)-[p:PARTICIPATED_IN]-(t:Tournament) where t.year = 2015 RETURN a
'''

with driver.session() as graphDB_Session:
    result = graphDB_Session.run(query).data()
df = pd.DataFrame(result)
df

2. i nomi delle squadre che hanno partecipato al torneo nel 2015 (farlo in un altro modo)

In [ ]:
query = ''' 

MATCH (a:Team)-[p:PARTICIPATED_IN]-(t:Tournament {year: 2015}) RETURN a
'''
with driver.session() as graphDB_Session:
    result = graphDB_Session.run(query).data()
df = pd.DataFrame(result)
df

3. il numero di partecipazioni di ogni nazione

In [17]:
query = ''' 

MATCH (a:Team)-[:REPRESENTS]-(p:Person)
RETURN a as Nazione, count(p) as Persone 
'''
with driver.session() as graphDB_Session:
    result = graphDB_Session.run(query).data()
df = pd.DataFrame(result)
df

,Nazione,Persone
0,"{'name': 'Equatorial Guinea', 'id': '1888631'}",21
1,"{'name': 'Nigeria', 'id': '1882893'}",94
2,"{'name': 'Australia', 'id': '1882891'}",90
3,"{'name': 'Japan', 'id': '1883723'}",93
4,"{'name': 'Korea DPR', 'id': '1883726'}",67
5,"{'name': 'England', 'id': '1883720'}",67
6,"{'name': 'France', 'id': '1884761'}",62
7,"{'name': 'Germany', 'id': '1882879'}",92
8,"{'name': 'Norway', 'id': '1882882'}",100
9,"{'name': 'Sweden', 'id': '1882883'}",104


4. gli allenatori dell'Italia

In [23]:
query = ''' 

MATCH (p:Person)-[:COACH_FOR]-(s:Squad)
WHERE s.id STARTS WITH 'Italy'
RETURN p
'''
with driver.session() as graphDB_Session:
    result = graphDB_Session.run(query).data()
df = pd.DataFrame(result)
df

,p
0,"{'name': 'Guenza Sergio', 'id': '36076', 'dob'..."
1,"{'name': 'Facchin Carlo', 'id': '184055', 'dob..."
2,"{'name': 'Milena Bertolini', 'id': '405427'}"


5. le giocatrici dell'Italia che hanno segnato e in quale partita hanno segnato (data)

In [ ]:
query = ''' 

MATCH (p:Person)-[:COACH_FOR]-(s:Squad)
WHERE s.id STARTS WITH 'Italy'
RETURN p
'''
with driver.session() as graphDB_Session:
    result = graphDB_Session.run(query).data()
df = pd.DataFrame(result)
df